# Endpoints 

# Rotas Simples com Express 🌿
- No expres, rotas seguem a seguintes estrutura: `app.METHOD(PATH, HANDLER)`. `METHOD` é um método HTTP em minísculo.
- `PATH` é um caminho relativo no servidor(pode ser uma string, ou até mesmo uma expressão regular);
- `HANDLER` é uma função que o Express chama quando a rota é encontrada.
- Handlers tem um formato como `function(req, res) {...}`, no qual `req` é o objeto de requisição, e `res` é o objeto de resposta.
- Tem uma variável node global que faz cálculo de path: `__dirname`.
- O Express analisa rotas de cima para baixo e executa o handler para a primeira correspondência.
- **Middleware**: são funções que interceptam o handler da rota, adicionando algum tipo de informação. Essas funções aceitam 3 argumentos: o objeto request, o objeto response e a próxima função no ciclo de `request-response` da aplicação. 
- Enquanto que o HTML serve HTML, uma API serve dados.
- Uma APIREST (REpresentational State Transfer) permite exportação de dados de uma forma simples, sem que os clientes precisem saber detalhes sobre o servidor.
- O arquivo `.env` fica oculto do GIT e é usado para passar variáveis de ambiente para a aplicação.
- **Requisições**: quando um cliente faz uma requisição, ele está enviando um pacote com: uma url, um tipo de ação HTTP (get, post, put, delete), body em json, metadados (no header com informações extras de autenticação etc).
- 

----

# Coisas Assíncronas na Programação para Web
Estudo e aplicação das estruturas async e await.

- V8 engine compila javascript para código de máquina.
- Node.js é um programa escrito em C++.

```mermaid
flowchart LR
A[JavaScript] --> B["Node.js (V8)"]
B --> C[Código de máquina]
```

[Callbacks to Promises: Promisifying Functions Mastery](https://krython.com/tutorial/typescript/callbacks-to-promises-promisifying-functions)
https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Promise#description

- **A Promise é um objeto que representa o sucesso ou a falha de uma operação assíncrona e seu valor resultante.**
- Uma função que retornar uma promise, ao invés de retornar o valor final, retorna um representante disse (a promise) que vai fornecer o valor futuramente. 
- As promises existem porque nem tudo dentro de um código é síncrono, nem tudo vai acontecer no mesmo instante em que é chamado para execução. E, nós não queremos que o código inteiro fique aguardando uma coisa ser executada para poder prosseguir. Isso sucede muito na programação para web por causa dos servidores, que podem estar em locais muito diferentes e que, então, vão ter seus tempos de resposta. Por tal motivo, precisamos expressar que algumas coisas dentro do código podem acontecer em paralelo com outras para darmos a trativa certa quando tudo for finalizado.
- aaaaaaahhh eu finalmente saquei!!!!!!!!!!!!! "Promise" vem de tipo "Prometo que em algum momento isso vai acontecer (ou vai dar errado), só não é agora, mas vai acontecer, será executado!". 
- O `resolve` é um método que você para expressar que deu tudo certo. Já o `reject` você usa para expressar que deu errado.

In [ ]:
const a = 5;
const b = 4;
let resultado;

const soma = (a, b) => {
    return new Promise((resolve, reject) => {
        setTimeout(() => {
            resolve(a + b);
        })}, 2000)
        // Essa promessa demora 2 segundos para resolver.
} 

// o then() vai buscar o valor quando a promise for resolvida.
// o then() recebe uma função de callback que vai ser executada quando a promise for resolvida.
// Já o catch() a gente chama para "pegar" o erro e tratar, caso a promise seja rejeitada. Ele também recebe uma função de callback. 
soma(a, b).then(
    console.log(`Resultado: ${resultado}`)
).catch(err => {
    console.log(`Erro: ${err}`)
});


Resultado: undefined


Promise { <pending> }


- O `fetch()` é bem importante. Ele serve para disparar uma requisição HTTP, isto é, solicitar alguma informação que está em outro servidor.
- Com o `async` você expressa que algo dentro da função é assíncrono; e com `await` você espera algo ser executado.
- Dentro do javascript, toda função que recebe um `async` na frente automaticamente vira uma `Promise`.

In [ ]:
async function buscaDados () {
    try {
        const response = await fetch("https://api.github.com/users/SorayaFerreira");
        const body = await response.json();
        
        console.log(body);

        return body.name;

    } catch (err) {
        console.log(err);

    } finally {
        console.log("foi");

    }
    

    // fetch("https://api.github.com/users/SorayaFerreira")
    // .then(response => {
    //     return response.json();
    // })
    // .then(body => {
    //     console.log(body);
    // })
    // .catch(err => {
    //     console.log(err);
    // })
    // .finally(() = >
    //     console.log("Foi;");
    // )
}

buscaDados().then(name => {
    console.log(name);
});

// Abaixo: não funciona porque a função buscaDados é assíncrona e retorna uma Promise.
// O console.log(name) vai ser executado antes da Promise ser resolvida, então ele
// vai imprimir uma Promise pendente, não o valor que você espera.
// console.log(name);

O código a seguir é o arquivo completo de um endpoint que quero entender.

In [ ]:
import type { APIRoute } from 'astro';
import { encodeBase32LowerCaseNoPadding } from "@/libs/authn/util";
import { clientRepository, authorizationRequestRepository } from '@/libs';
import { sessionProvider } from '@/libs/authn';

export const GET: APIRoute = async ({ cookies, url, redirect }) => {
  const sessionToken = cookies.get('session')?.value;

  if (!sessionToken) {
    return redirect(`/login?redirect_uri=${encodeURIComponent(url.toString())}`);
  }

  const { session } = await sessionProvider.validateSessionToken(sessionToken);

  if (!session) {
    return redirect(`/login?redirect_uri=${encodeURIComponent(url.toString())}`);
  }

  const { user_id } = session

  const requestData = Object.fromEntries(url.searchParams.entries())

  const { response_type, client_id } = requestData

  const client = clientRepository.get({
    id: client_id
  })

  if (!client) {
    return new Response(null, { status: 400 })
  }

  const { callback_endpoint } = client;

  switch (response_type) {
    case "code": {
      const { code_challenge, code_challenge_method } = requestData

      if (code_challenge_method.toLowerCase() !== "s256") {
        return new Response(null, { status: 400 })
      }

      const code = generateAuthorizationCode()

      authorizationRequestRepository.create({
        code,
        client_id,
        project_id: client.project_id,
        code_challenge,
        user_id
      });

      return redirect(`${callback_endpoint}?code=${code}`);
    }
    default: {
      return new Response(null, { status: 400 })
    }
  }
}

function generateAuthorizationCode() {
  const bytes = new Uint8Array(32);
  crypto.getRandomValues(bytes);

  return encodeBase32LowerCaseNoPadding(bytes)
}

O endpoint acima faz parte de uma etapa do provider de um serviço de autenticação oferecido pelo MVP do Bufunfa, o "logar com o Bufunfa". Ele:
- Recebe a solicitação de autorização do CentoBank.
- Se o usuário não estiver logado, ele redireciona para a tela de login, na qual deve ser logado, e redirecionado de volta para o fluxo de autorização (vejam o comportamento do "Login com Google" para detalhes)
- Gera um código de autenticação (code randomico).
- Armazena no banco de dados, na tabela oauth_authorization_request, o code, client_id, code challenge e o user_id.
- Redireciona o usuário para o callback_endpoint criado para o Cento Bank, passando o code gerado.

Nesse sentido, a missão desse notebook é destrinchar todo esse código, objetivando boa compreensão de rotas feitas com TypeScript.

Os blocos de código que seguem abaixo são explicados linha a linha.

```typescript
import type { APIRoute } from 'astro'; 
import { encodeBase32LowerCaseNoPadding } from "@/libs/authn/util";
import { clientRepository, authorizationRequestRepository } from '@/libs';
import { sessionProvider } from '@/libs/authn';
```

```typescript
export const GET: APIRoute = async ({ cookies, url, redirect }) => {
  const sessionToken = cookies.get('session')?.value;
```

```typescript
  if (!sessionToken) {
    return redirect(`/login?redirect_uri=${encodeURIComponent(url.toString())}`);
  }

  const { session } = await sessionProvider.validateSessionToken(sessionToken);
```

```typescript
  if (!session) {
    return redirect(`/login?redirect_uri=${encodeURIComponent(url.toString())}`);
  }

  const { user_id } = session

  const requestData = Object.fromEntries(url.searchParams.entries())

  const { response_type, client_id } = requestData
```

```typescript
  const client = clientRepository.get({
    id: client_id
  })

  if (!client) {
    return new Response(null, { status: 400 });
  }

  const { callback_endpoint } = client;
```

```typescript
  switch (response_type) {
    case "code": {
      const { code_challenge, code_challenge_method } = requestData

      if (code_challenge_method.toLowerCase() !== "s256") {
        return new Response(null, { status: 400 })
      }

      const code = generateAuthorizationCode()

      authorizationRequestRepository.create({
        code,
        client_id,
        project_id: client.project_id,
        code_challenge,
        user_id
      });

      return redirect(`${callback_endpoint}?code=${code}`);
    }
    default: {
      return new Response(null, { status: 400 })
    }
  }

```

```typescript
function generateAuthorizationCode() {
  const bytes = new Uint8Array(32);
  crypto.getRandomValues(bytes);

  return encodeBase32LowerCaseNoPadding(bytes)
}
```

### precisa adaptar

```javascript
const code_verifier = new TextEncoder().encode(atob(code_verifier_cookies));
```

- `code_verifier_cookies` é uma variável que contém uma string codificada em Base64. Este nome sugere que a string foi obtida a partir de cookies armazenados no navegador.

- `atob(code_verifier_cookies)`:
   - Função `atob`: `atob` é uma função JavaScript que decodifica uma string Base64 para texto ASCII. `atob` significa "ASCII to binary".
   - Processo: A string codificada em Base64 (que está armazenada em `code_verifier_cookies`) é decodificada para obter o texto original. 
   - Resultado: O resultado de `atob(code_verifier_cookies)` é uma string em texto plano.

- `new TextEncoder().encode(...)`:
   - `TextEncoder`: `TextEncoder` é uma interface da API de Codificação de Texto que permite converter uma string de texto em um buffer de dados binários codificado em UTF-8.
   - Método `encode`: O método `encode` da `TextEncoder` converte uma string de texto para um `Uint8Array` de bytes.
   - Processo: A string de texto obtida da decodificação Base64 é convertida em uma sequência de bytes codificada em UTF-8.

- Atribuição à Constante `code_verifier`:
   - O resultado final da codificação UTF-8 (um `Uint8Array` de bytes) é armazenado na constante `code_verifier`.

---

# Results.ts

Abaixo é apresentado um exemplo de implementação da abordagem mencionada em [**try... catch VS Results**](./tests.md#trycatch--results).
- Link para o [playground do exemplo](https://www.typescriptlang.org/play/?#code/FAYw9gdgzgLgBAJwKZQK4Bt4F44CUUYwB0A7ggIYAOAFNQJRxYB8cAZqhCOWALLkAiYAArpyATzIBLAOYALGP0lRK5CgFEECMPTrBg5KGM5sOIGJMgnO3PoJHipchUpXrN2ugC44QrQFslJAAeWARJCGkWAG9gODgYWS0SOAgkZI0tBGoAIgBlJD9KZDgAEwBDuCR3bN0AXz12TnNLAAkkdHRtCHI-JG9Q8OkvPAJMEJgwiIAaOAywBGjYuElWOGpu3sYsHGyauBi4uOQYVAQIEbRMIgz1tNn3LOyCyhgxFJ6kGt04+qXwaHgvSgUHI0iQjDgAAMWpIZgASKIbJC1IhwADq7XAvUheiOSBOZwuhCIAHkANbUIEgsF1PTAAD09LgAFpWWz2RzOVzWXTXpRwXwxAAjJC+MABKDBAAqLBwYol0pYAB84FK9EgAB6UebwPng-CXGBBKUzNSyolXcnG5UW4gZIJm9VanXvIEqED60bwA6Hf6wRBe3JiPxCsDoCFBkNhnLIQ01XGHTXahC6sT8uBWmUQn2HQ4AbVjhEjofQAF1vNkwGTsktc3EAG7kdCoPqq2twX65pMuvX3BAO805usFwPBkvluBPTTZKbt3ON5uttTtzuJ50pqxmCznTOmiGpetVJjUBct7xS4YGwjG02LOuHY6nc7UKJwEeG4thieV6szU-g2pvkOVc4m7DdGi3Sx7TUGYpX3JBDwWE8mzPWZLy9G9Zjve8AwJZ9X3fIsxy-CsqgQGc4H-DsgJ+BNQPXeAIOac4lF3LDqELTBvCvMYTSw4ZOPgJQMzJa19jnB98SfANDUIzBPzLLYdirGtcxAyoGM3Zjlige0+LNDivW4jD9KYASvR0vsB3EnDcOkwS5JgBTSyUycyNU4C6I05NGNMbSyCoTC1AhDgyQgMASAgY9WAgbx6EYFh5UCa1hiSyUgh4o1TKYdswN8pptzgALKCCkKIDCiKouoGK4oYZhVXQw0gpyrtNKYwritKnBQvCyLotitY6pYQURTSxVhhG0V-GSzLmuw+8JjeIdbL9eB-xJBAxohGL6C82yVjWdbNumyVlgBchOCQMBVjGhhltsyS8MolCkA2saiASJAIGock6CILgYBAWRqAyGiHvU2zH0JclkMXN6TqQMG1LgAGgbWMi7ok+8ofOG4yLgAw0KxjsVy8vKtMKjhOv0+DEOPIpxUCbwxoykzbzM5mEetXK2r8ymIGpvccAPI9DMNYymtM4Y1VanyKcsKmKBKmnhYQ0XBPhxnJW8SaWdm7KJvEUauaze64gOsXCE1hUztgC6PWunwEcxh7Hvsr1rcCD7ZC+2hBKGmzXbrC2lBuf2XaD2yEiSGTiX-YnbIhoOcdjq548jwDiaT83VmoUPNEtzBPclOgI9d6OItTmBi6QIh05w7O7MJDXjq12v69o+ogA).

In [ ]:
// deno-lint-ignore-file no-namespace

const result = Result.wrap(() => funcaoMaDoPlaywrightDisparaErro())

async function funcaoMaDoPlaywrightDisparaErro(): Promise<string> {
  throw new Error("Sempre dá erro")
}

function Hello(name: string): Result<string, Error> {
  if (name === "") {
    return Result.Err(new Error("empty name"))
  }

  const message = `Hi, ${name}. Welcome`

  return Result.Ok(message)
}


// ------------------------------


type MaybePromise<T> = Promise<T> | T

export type Result<T, E> = Result.Ok<T> | Result.Err<E>

export namespace Result {
    const resultSymbol = Symbol("result")

    export type Ok<T> = {
        [resultSymbol]: "ok"
        value: T
    }

    export type Err<E> = {
        [resultSymbol]: "err",
        value: E
    }

    export function Ok<T, E = never>(value: T): Result<T, E> {
        return ({ [resultSymbol]: "ok", value })
    }

    export function Err<E, T = never>(value: E): Result<T, E> {
        return ({ [resultSymbol]: "err", value })
    }

    export function isOk<T, E>(result: Result<T, E>): result is Ok<T> {
        return result[resultSymbol] === "ok"
    }

    export function isErr<T, E>(result: Result<T, E>): result is Err<E> {
        return result[resultSymbol] === "err"
    }

    export function wrap<T, E = unknown>(fn: () => Promise<T>): Promise<Result<T, E>>
    export function wrap<T, E = unknown>(fn: () => T): Result<T, E>
    export function wrap<T, E = unknown>(fn: () => MaybePromise<T>): MaybePromise<Result<T, E>> {
        try {
            const valueOrPromise = fn()

            if (valueOrPromise instanceof Promise) {
                return valueOrPromise.then(Ok).catch(Err)
            }

            return Ok(valueOrPromise)
        } catch (err) {
            return Err(err as E)
        }
    }

    export function unwrap<T, E = never>(promise: Promise<Result<T, E>>): Promise<T>
    export function unwrap<T, E = never>(result: Result<T, E>): T
    export function unwrap<T, E = never>(resultOrPromise: MaybePromise<Result<T, E>>): MaybePromise<T> {
        if (resultOrPromise instanceof Promise) {
            return resultOrPromise.then((result) => {
                if (isErr(result)) {
                    throw result.value
                }

                return result.value
            })
        }

        if (isErr(resultOrPromise)) {
            throw resultOrPromise.value
        }

        return resultOrPromise.value
    }
}
